# Automatisierte Vorverarbeitung

Das Ziel dieses Jupyter Notebook ist es die Vorverarbeitung eines rohen Datensatz automatisiert durchzufühen. Damit die einzelnen Schritte in einer Pipline abgearbeitet werden können. Die folgenden Schritte werden ausgeführt.  

1. Konkatenieren der rohen Merkmal CSV Dateien
2. Bildaufnahmen zuschneiden und skalieren
3. Überbelichtete und unterbelichtete Bilder aus dem Datensatz entfernen.
4. Messwertreihen aus der Merkmal CSV kopieren
5. Zusätzliche Merkmale aus den bestehenden berechnen 
6. Messwertreihen als eigenstädige CSV Dateien und veränderte Merkmal CSV mit den berechneten Merkmalen abspeichern als CSV Datei

In [6]:
import pandas
import os
# Regex Vergleiche
import re
import numpy as np
import cv2
import math

In [7]:
# Pfad zu den Datensatz
dataset = 53
path = "E:/bachelor/daten/" + str(dataset) + "/2018_7_9_13_49/" 
outputDir = "E:/bachelor/daten/" + str(dataset) + "/zugeschnitten/"
if not os.path.exists(outputDir):
  os.makedirs(outputDir)

In [8]:
##### 1
# Lesen der Merkmal CSV Dateien
files = os.listdir(path)
featuresDf = []
for file in files:
    if re.match('[0-9]', file) is not None:
        featuresDf.append(pandas.read_csv(filepath_or_buffer= path + file + "/merkmaleRoh.csv"))  
# Jeder Panda Dataframe wird konkateniert
length = len(featuresDf)
if length >= 1:
    result = featuresDf[0]
    for df in featuresDf[1:]:
        frames = [result, df]
        result = pandas.concat(frames, ignore_index=True)
featuresDf = result

In [9]:
##### 2
# Die Funktion schneidet und skaliert einen Datensatz von Bildern
# @param path != null / String Pfad zu den Bildern des Datensatz
#  outputDir String Pfadname des Ausgabeordners
# @return Liste mit den Pfaden zu allen zugeschnittenen Bildern
def cropAndScaleImages(path, outputDir):
    #y = 85
    #x = 477
    ##### Für Datensatz 53 mit MTB
    y = 40 
    x = 400
    # Breite und Höhe des Ausschnits
    h = 1475
    w = 280
    # Größe des neuen Bild
    scaleFactor = 0.25 # 0.26 ist Skalierung angepasst für Xception
    wNew = w * scaleFactor
    hNew = h * scaleFactor
    i = -1
    filesImageNumber = []
    dirs = os.listdir(path)
    for dirNumber in dirs:
        filesImageNumber.append(os.listdir(path + dirNumber))
    for files in filesImageNumber:
        i = i + 1  
        outputDirName = outputDir + dirs[i] 
        print(outputDirName)
        if not os.path.exists(outputDirName):
            os.makedirs(outputDirName)
        for name in files:
            if "jpg" not in name:
                continue
            img = cv2.imread(path + dirs[i] + "/" + name)
            img = img[y:y+h, x:x+w]
            img = cv2.resize(img,(int(wNew),int(hNew))) # Bild wird skaliert
            cv2.imwrite( outputDirName + "/" + name, img) 
    # Liste mit den Pfaden zu den Bilder zurückgeben
    imagePaths = []
    folders = os.listdir(outputDir)
    folders = sorted(folders, key=int) #sortiert die Reihenfolge de Ordner aufsteifend
    print("Bilder aus folgenden Ordnern werden geladen: " + str(folders))
    for folder in folders: # Aus der Liste der Ordner wird ein Ordner ausgewählt
        filesPath = outputDir + folder + "/"
        files = os.listdir(filesPath)
        print("Ordner der geladen wird: " + str(folder))
        for name in files: # Ein Dateiname aus diesem Ordner
            if "jpg" not in name:
                continue
            imagePaths.append(filesPath + name) # Pfad zu den Bild wird zur Liste mit Pfaden hinzugefügt      
    return imagePaths

In [10]:
imagePaths = cropAndScaleImages(path, outputDir)

E:/bachelor/daten/53/zugeschnitten/1
E:/bachelor/daten/53/zugeschnitten/2
E:/bachelor/daten/53/zugeschnitten/3
E:/bachelor/daten/53/zugeschnitten/4
E:/bachelor/daten/53/zugeschnitten/5
Bilder aus folgenden Ordnern werden geladen: ['1', '2', '3', '4', '5']
Ordner der geladen wird: 1
Ordner der geladen wird: 2
Ordner der geladen wird: 3
Ordner der geladen wird: 4
Ordner der geladen wird: 5


In [11]:
##### 3
# Berechne Indexe für Bilder die unter oder überbelichtet sind
indexe = []
imagePathsToDelete = [] # String Pfade zu den Bildern welche gelöscht werden 
i = 0
upperThreshold = 204 # obere Grauwert Schwelltwert für überbelichtete Bilder
lowerThreshold = 50  # unterer Grauwert Schwellwert für unterbelichtete Bilder
for path in imagePaths:
    img = cv2.cvtColor(cv2.imread(path),cv2.COLOR_BGR2GRAY)
    img = img.astype('float32')
    imgFlat = img.ravel()
    mean = imgFlat.sum() / imgFlat.size
    if mean > upperThreshold or mean < lowerThreshold:
        indexe.append(i)
        imagePathsToDelete.append(imagePaths[i])
    i = i + 1
# merkmaleRoh.csv Zeilen mit gefundenen Index löschen
featuresDf.drop(indexe, inplace=True)
# Die Inecies müssen zurückgesetzt werden nach den Aufruf von drop. Ansonsten kommte es zu KeyValue Fehlern
featuresDf.reset_index(drop=True,inplace=True)

In [12]:
for img in imagePathsToDelete:
    print(img)
    os.remove(img)
imagePathsToDelete.clear()

E:/bachelor/daten/53/zugeschnitten/3/1533817114855.jpg


In [13]:
##### 4
accYList = featuresDf['Y-Achse Beschleunigungswerte in m/s^2'].str.split(" ").tolist()
accList = []
accList.append(pandas.DataFrame(np.array(accYList[0]).astype(float)).transpose())
for x in accYList[1:]:
    accList.append(pandas.DataFrame(np.array(x).astype(float)).transpose())
accYDf = pandas.concat(accList, ignore_index=True)
# Konvertiere Beschleunigungssensordaten Z-Achse zu einem DataFrame
measuredDataList = featuresDf['Z-Achse Beschleunigungswerte in m/s^2'].str.split(" ").tolist()
accList = []
accList.append(pandas.DataFrame(np.array(measuredDataList[0]).astype(float)).transpose())
for x in measuredDataList[1:]:
    accList.append(pandas.DataFrame(np.array(x).astype(float)).transpose())
accZDf = pandas.concat(accList, ignore_index=True)
# Konvertiere Nick Messwerte zu DataFrame
measuredDataList = featuresDf['Nick Messwerte in rad'].str.split(" ").tolist()
pitchList = []
pitchList.append(pandas.DataFrame(np.array(measuredDataList[0]).astype(float)).transpose())
for x in measuredDataList[1:]:
    pitchList.append(pandas.DataFrame(np.array(x).astype(float)).transpose())
nickDf = pandas.concat(pitchList, ignore_index=True)
measuredDataList = featuresDf['Zeitstempel der Messwerte in ns'].str.split(" ").tolist()
timestampsList = []
timestampsList.append(pandas.DataFrame(np.array(measuredDataList[0]).astype(float)).transpose())
for x in measuredDataList[1:]:
    timestampsList.append(pandas.DataFrame(np.array(x).astype(float)).transpose())
sensorTimestampsDf = pandas.concat(timestampsList, ignore_index=True)

In [14]:
##### 5
def calcVariance(meansDf, dfValues):
    variance = []
    for i, values in dfValues.iterrows():
        sum = 0
        for value in values:
            if math.isnan(value) == False:
                tempDifference = value-meansDf[i]
                sum += tempDifference * tempDifference
        variance.append("{0:.5f}".format(round(sum / values.count(),5)))
    return np.array(variance).astype(float)
def calculateStandardDeviation(varianceDf):
    deviation = []
    for v in varianceDf:
        temp = math.sqrt(np.abs(v))
        deviation.append("{0:.5f}".format(round(temp,5)))
    return np.array(deviation).astype(float)

In [15]:
# Neue Spalten werden erstellt und Mittelwerte zugewiesen
featuresDf['Mittelwert Y-Achse Beschleunigung in m/s^2'] = accYDf.T.mean()
featuresDf['Mittelwert Z-Achse Beschleunigung in m/s^2'] = accZDf.T.mean()
featuresDf['Mittelwert Nick in rad'] = nickDf.T.mean()
# Berechnung der Varianz und hinzufügen der Ergebnisse als neue Spalte in dem DataFrame 
featuresDf['Varianz Y-Achse Beschleunigung in m/s^2'] = calcVariance(featuresDf['Mittelwert Y-Achse Beschleunigung in m/s^2'],accYDf)
featuresDf['Varianz Z-Achse Beschleunigung in m/s^2'] = calcVariance(featuresDf['Mittelwert Z-Achse Beschleunigung in m/s^2'],accZDf)
featuresDf['Varianz Nick in rad'] = calcVariance(featuresDf['Mittelwert Nick in rad'],nickDf)
# Berechne Standardabweichung
featuresDf['Standardabweichung Y-Achse Beschleunigung in m/s^2'] = calculateStandardDeviation(featuresDf['Varianz Y-Achse Beschleunigung in m/s^2'])
featuresDf['Standardabweichung Z-Achse Beschleunigung in m/s^2'] = calculateStandardDeviation(featuresDf['Varianz Z-Achse Beschleunigung in m/s^2'])
featuresDf['Standardabweichung Nick in rad'] = calculateStandardDeviation(featuresDf['Varianz Nick in rad'])

In [16]:
##### 6
# Änderungen in CSV Datei schreiben
outputFolder = '../daten/merkmale_datensatz_' + str(dataset) + '/optimiertXception'
if not os.path.exists(outputFolder):
  os.makedirs(outputFolder)
featuresDf.set_index('Zeitstempel in Unixzeit', inplace=True)
featuresDf.to_csv( outputFolder + '/merkmale.csv')
# Speichert die DatenFrames mit den Messwerten in csv Dateien
accYDf.index.names = ["index"]
accYDf.to_csv(outputFolder + '/yAchseBeschleunigungswerte.csv')
accZDf.index.names = ["index"]
accZDf.to_csv(outputFolder + '/zAchseBeschleunigungswerte.csv')
nickDf.index.names = ["index"]
nickDf.to_csv(outputFolder + '/nickMesswerte.csv')
sensorTimestampsDf.index.names = ["index"]
sensorTimestampsDf.to_csv(outputFolder + '/sensoreventZeitstempel.csv')